# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "output_data/weather_df.csv"
# output_data_file

types_df = pd.read_csv(output_data_file)
weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df.head()

,Cities,Lat,Lng,Country,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,Walvis Bay,-22.96,14.51,NaN,1587049139,6,52,69.80,9.17
1,Bugene,-1.59,31.14,TZ,1587049139,65,57,70.52,3.65
2,Mataura,-46.19,168.86,NZ,1587049140,97,90,44.01,3.00
3,Troitsko-Pechorsk,62.71,56.20,RU,1587049140,100,78,36.36,8.63
4,Topaowan,30.67,112.69,CN,1587049140,95,73,69.13,12.86


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
# Use Lat and Lng as locations
location=weather_data_df[["Lat","Lng"]]
# Use humidity as the weigtht
humidity=weather_data_df["Humidity"].astype(float)
# Add Heatmap layer to map
fig =gmaps.figure()
heatmap_layer=gmaps.heatmap_layer(location, weights=humidity, dissipating =False, max_intensity=15, point_radius=2)
fig.add_layer(heatmap_layer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
Ideal_df=pd.DataFrame(weather_data_df, columns=["Cities", "Max Temp", "Cloudiness", "Wind Speed"])
max_temp=(Ideal_df["Max Temp"]<=80) &(Ideal_df["Max Temp"]>=70)
cloudiness = Ideal_df["Cloudiness"] == 0
Wind_Speed=Ideal_df["Wind Speed"] < 10
# Filter the ideal cities
Ideal_df[max_temp&cloudiness&Wind_Speed].head()
# Drop any rows that don't contain all three conditions. 
Ideal_df.dropna()

,Cities,Max Temp,Cloudiness,Wind Speed
0,Walvis Bay,69.80,6,9.17
1,Bugene,70.52,65,3.65
2,Mataura,44.01,97,3.00
3,Troitsko-Pechorsk,36.36,100,8.63
4,Topaowan,69.13,95,12.86
...,...,...,...,...
571,Maba,68.02,0,3.04
572,Thinadhoo,85.33,96,6.35
573,Constantine,78.80,75,3.36
574,Qandala,85.96,4,5.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame(weather_data_df, columns=["Cities", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"]=""
hotel_df.head()

,Cities,Country,Lat,Lng,Hotel Name
0,Walvis Bay,NaN,-22.96,14.51,
1,Bugene,TZ,-1.59,31.14,
2,Mataura,NZ,-46.19,168.86,
3,Troitsko-Pechorsk,RU,62.71,56.20,
4,Topaowan,CN,30.67,112.69,


In [7]:
# Set parameters to search for hotels with 5000 meters.
target_coordinates=f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search="hotel"
target_radius=5000
target_type="hotel"
# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Performing a GET Request 
response = requests.get(base_url,params=params)
hotel_data = response.json()
print(json.dumps(hotel_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "CrQCIQEAANHKGWPa13OAc10M2kIWMZ89L1xlBQQKoL44_0qA-cm2dg4wSdZr9LN29ZeV6RElyc0VPz_HD7fgdKuri6hkg3RUBr_DO1Fl_73xqVA7xg-nhEbjkFVWI-vgLXAFGshJDzprU4C5LHfOi87hutojhqioiOTg55Jgp_71fYwiYmewTQtPta34Fsnm3wRzfNZWZvz2S4Q0D1w_WzNRr4_RmV8m4OZLRNOegy-rQcmOUQmSZmx06eDHQlUHIYmRDEMns4pwTCS95QaDDdYj7GqYR9pWBTwov6sJfzCO5gdsvXTy3nSD6NcqTBPfNDexbkF2SjFx7lcDm0jI_Bimxwvo5VTuAmwLZGHWrt5diKH0IusMKiUQ7k1I3IndZD8gytbeFLW1BxYhGWvF_bwIeMqaQXoSELtd_7lBDSQUPUkld-ngxE0aFD_1S4FwIjA2ZphsuDfmtcPymIjl",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -22.961277,
                    "lng": 14.483048
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.95981387010728,
                        "lng": 14.48436582989272
                    },
                    "southwest": {
                        "lat": -22.96251352989272,
                      

In [8]:
# Store the first Hotel result into the DataFrame.
hotel_df.iloc[[0],[4]]

,Hotel Name
0,


In [16]:
# creates a list to store hotel names
hotel_name=[]
# parameters dictionary
params={
    "radius":5000,
    "types":"hotel",
    "keyword":"hotel",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
            hotel_df.iloc[index, 4] = name_address['results'][0]['name']
            hotel_name.append(name_address['results'][0]['name'])
    except  IndexError:
         hotel_name.append(np.nan)   
    print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/

KeyboardInterrupt: 

In [18]:
name_address = name_address.json()
print(json.dumps(name_address, indent=4, sort_keys=True))


{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 61.218163,
                    "lng": 21.6730049
                },
                "viewport": {
                    "northeast": {
                        "lat": 61.21955177989272,
                        "lng": 21.67432852989272
                    },
                    "southwest": {
                        "lat": 61.21685212010728,
                        "lng": 21.67162887010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png",
            "id": "97696705202cb68109ebe90166042ac24ff34afa",
            "name": "Vuojokis\u00e4\u00e4ti\u00f6",
            "place_id": "ChIJX8pyXKV7iUYRAX6ZWXd5ksU",
            "plus_code": {
                "compound_code": "6M9F+76 Eurajoki, Finland",
                "global_code": "9GH36M9F+76"
            },


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display Map
fig